# 8.0.4 人机协作与时间回溯

本节将通过两个实战案例，深入学习 LangGraph 的两个高级特性：

1. **Human-in-the-Loop (HITL)**：让人类参与 AI 决策过程
2. **Time Travel**：回溯到历史状态，探索不同的执行路径

这两个特性是构建可靠、可控 AI 系统的关键能力。

---

# 案例一：智能翻译审核系统（HITL）

## 场景描述

我们要构建一个智能翻译系统，但涉及敏感内容（如法律条款、医学术语）时，
需要人工审核后才能输出最终结果。

## 工作流程

```
用户输入 → 翻译内容 → [人工审核] → 输出结果
                          ↓
                     拒绝 → 终止
```

In [ ]:
# ========================================
# 1. 导入依赖
# ========================================

import os
from typing import Literal, TypedDict, Annotated
from operator import add

from langchain_core.messages import AnyMessage, HumanMessage, AIMessage
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.constants import START, END
from langgraph.graph import StateGraph
from langgraph.types import interrupt, Command

In [ ]:
# ========================================
# 2. 初始化大模型
# ========================================

llm = ChatOpenAI(
    model="gpt-4o-mini",
    api_key=os.environ.get("OPENAI_API_KEY"),
)

In [ ]:
# ========================================
# 3. 定义状态结构
# ========================================

class TranslationState(TypedDict):
    """翻译系统的状态定义"""
    source_text: str           # 原文
    source_lang: str           # 源语言
    target_lang: str           # 目标语言
    translated_text: str       # 翻译结果
    is_sensitive: bool         # 是否包含敏感内容
    review_passed: bool        # 是否通过审核
    final_output: str          # 最终输出

In [ ]:
# ========================================
# 4. 定义翻译节点
# ========================================

def translate_text(state: TranslationState) -> TranslationState:
    """调用 LLM 进行翻译"""
    prompt = f"""请将以下{state['source_lang']}文本翻译成{state['target_lang']}：

{state['source_text']}

只输出翻译结果，不要添加任何解释。"""
    
    response = llm.invoke(prompt)
    translated = response.content
    
    # 简单判断是否包含敏感词（实际应用中会更复杂）
    sensitive_keywords = ["法律", "医疗", "金融", "合同", "药物", "legal", "medical", "financial"]
    is_sensitive = any(kw in state['source_text'].lower() or kw in translated.lower() 
                       for kw in sensitive_keywords)
    
    return {
        "translated_text": translated,
        "is_sensitive": is_sensitive
    }

In [ ]:
# ========================================
# 5. 定义人工审核节点（核心 HITL 逻辑）
# ========================================

def human_review(state: TranslationState) -> Command[Literal["output_result", "__end__"]]:
    """
    人工审核节点 - 使用 interrupt() 暂停执行
    
    这是 HITL 的核心：
    1. interrupt() 会暂停 Graph 执行
    2. 返回审核请求给用户
    3. 用户通过 Command(resume=True/False) 继续执行
    """
    
    # 使用 interrupt 暂停执行，等待人工审核
    review_result = interrupt({
        "type": "translation_review",
        "message": "请审核以下翻译结果",
        "original": state["source_text"],
        "translated": state["translated_text"],
        "warning": "检测到敏感内容，需要人工确认" if state["is_sensitive"] else None
    })
    
    # 根据审核结果决定下一步
    if review_result:
        return Command(goto="output_result", update={"review_passed": True})
    else:
        return Command(goto=END, update={"review_passed": False, "final_output": "翻译已被拒绝"})

In [ ]:
# ========================================
# 6. 定义输出节点
# ========================================

def output_result(state: TranslationState) -> TranslationState:
    """输出最终翻译结果"""
    return {
        "final_output": state["translated_text"]
    }

In [ ]:
# ========================================
# 7. 定义路由逻辑
# ========================================

def should_review(state: TranslationState) -> Literal["human_review", "output_result"]:
    """
    判断是否需要人工审核
    - 敏感内容 → 需要审核
    - 普通内容 → 直接输出
    """
    if state["is_sensitive"]:
        return "human_review"
    return "output_result"

In [ ]:
# ========================================
# 8. 构建 Graph
# ========================================

builder = StateGraph(TranslationState)

# 添加节点
builder.add_node("translate", translate_text)
builder.add_node("human_review", human_review)
builder.add_node("output_result", output_result)

# 添加边
builder.add_edge(START, "translate")
builder.add_conditional_edges("translate", should_review)
builder.add_edge("output_result", END)

# 编译（必须使用 checkpointer 才能支持 interrupt）
checkpointer = InMemorySaver()
translation_graph = builder.compile(checkpointer=checkpointer)

In [ ]:
# 可视化图结构
from IPython.display import Image, display
display(Image(translation_graph.get_graph(xray=True).draw_mermaid_png()))

In [ ]:
# ========================================
# 9. 演示：普通翻译（无需审核）
# ========================================

normal_config = {"configurable": {"thread_id": "normal-translation"}}

result = translation_graph.invoke(
    {
        "source_text": "今天天气真好，适合出去散步。",
        "source_lang": "中文",
        "target_lang": "英文"
    },
    config=normal_config
)

print("【普通翻译 - 无需审核】")
print(f"原文: {result['source_text']}")
print(f"译文: {result['final_output']}")

In [ ]:
# ========================================
# 10. 演示：敏感内容翻译（需要审核）
# ========================================

sensitive_config = {"configurable": {"thread_id": "sensitive-translation"}}

# 第一步：提交翻译请求（会在审核节点暂停）
result = translation_graph.invoke(
    {
        "source_text": "根据本合同法律条款，乙方需在30日内支付全部款项。",
        "source_lang": "中文",
        "target_lang": "英文"
    },
    config=sensitive_config
)

print("【敏感内容翻译 - 等待审核】")
print(f"原文: {result['source_text']}")
print(f"初步译文: {result['translated_text']}")
print(f"检测到敏感内容: {result['is_sensitive']}")
print("\n⏸️ 系统已暂停，等待人工审核...")

# 查看中断信息
if '__interrupt__' in result:
    print(f"\n审核请求: {result['__interrupt__']}")

In [ ]:
# ========================================
# 11. 人工批准翻译
# ========================================

# 使用 Command(resume=True) 批准翻译
approved_result = translation_graph.invoke(
    Command(resume=True),
    config=sensitive_config
)

print("【审核通过】")
print(f"最终输出: {approved_result['final_output']}")
print(f"审核状态: {'通过' if approved_result['review_passed'] else '拒绝'}")

In [ ]:
# ========================================
# 12. 演示：人工拒绝翻译
# ========================================

reject_config = {"configurable": {"thread_id": "rejected-translation"}}

# 提交敏感翻译
result = translation_graph.invoke(
    {
        "source_text": "患者需每日服用该药物两次，请遵医嘱。",
        "source_lang": "中文",
        "target_lang": "英文"
    },
    config=reject_config
)

print("【医疗内容翻译 - 等待审核】")
print(f"初步译文: {result['translated_text']}")

# 人工拒绝
rejected_result = translation_graph.invoke(
    Command(resume=False),
    config=reject_config
)

print("\n【审核拒绝】")
print(f"最终输出: {rejected_result['final_output']}")

---

# 案例二：故事创作助手（Time Travel）

## 场景描述

我们要构建一个故事创作助手，它会：
1. 生成故事的开头
2. 根据开头生成故事的发展
3. 用户可以"回到过去"，从开头重新生成不同的发展

## Time Travel 的价值

- **探索可能性**：不满意当前结果？回到之前的节点重新尝试
- **调试定位**：发现问题时，回溯到问题发生前的状态
- **A/B 测试**：从同一个起点，探索不同的执行路径

In [ ]:
# ========================================
# 1. 导入依赖
# ========================================

import uuid
from typing import TypedDict
from typing_extensions import NotRequired
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.constants import START, END
from langgraph.graph import StateGraph
from langchain_openai import ChatOpenAI
import os

In [ ]:
# ========================================
# 2. 初始化大模型
# ========================================

story_llm = ChatOpenAI(
    model="gpt-4o-mini",
    api_key=os.environ.get("OPENAI_API_KEY"),
)

In [ ]:
# ========================================
# 3. 定义状态结构
# ========================================

class StoryState(TypedDict):
    """故事创作的状态定义"""
    theme: str                         # 故事主题
    opening: NotRequired[str]          # 故事开头
    development: NotRequired[str]      # 故事发展
    ending: NotRequired[str]           # 故事结局

In [ ]:
# ========================================
# 4. 定义故事创作节点
# ========================================

def generate_opening(state: StoryState) -> StoryState:
    """生成故事开头"""
    prompt = f"""请根据主题「{state['theme']}」，写一个故事的精彩开头。
要求：50字以内，设置悬念，吸引读者继续阅读。只输出故事内容。"""
    
    response = story_llm.invoke(prompt)
    return {"opening": response.content}


def generate_development(state: StoryState) -> StoryState:
    """生成故事发展"""
    prompt = f"""故事主题：{state['theme']}
故事开头：{state['opening']}

请续写故事的发展部分，80字以内。要有转折和冲突。只输出故事内容。"""
    
    response = story_llm.invoke(prompt)
    return {"development": response.content}


def generate_ending(state: StoryState) -> StoryState:
    """生成故事结局"""
    prompt = f"""故事主题：{state['theme']}
故事开头：{state['opening']}
故事发展：{state['development']}

请写一个出人意料又合情合理的结局，50字以内。只输出故事内容。"""
    
    response = story_llm.invoke(prompt)
    return {"ending": response.content}

In [ ]:
# ========================================
# 5. 构建 Graph
# ========================================

story_builder = StateGraph(StoryState)

# 添加节点
story_builder.add_node("opening", generate_opening)
story_builder.add_node("development", generate_development)
story_builder.add_node("ending", generate_ending)

# 添加边：开头 → 发展 → 结局
story_builder.add_edge(START, "opening")
story_builder.add_edge("opening", "development")
story_builder.add_edge("development", "ending")
story_builder.add_edge("ending", END)

# 编译（Time Travel 需要 checkpointer）
story_checkpointer = InMemorySaver()
story_graph = story_builder.compile(checkpointer=story_checkpointer)

In [ ]:
# 可视化图结构
from IPython.display import Image, display
display(Image(story_graph.get_graph(xray=True).draw_mermaid_png()))

In [ ]:
# ========================================
# 6. 第一次创作：生成完整故事
# ========================================

story_config = {
    "configurable": {
        "thread_id": str(uuid.uuid4())  # 使用唯一的会话 ID
    }
}

first_story = story_graph.invoke(
    {"theme": "一个程序员意外发现了时间旅行的秘密"},
    config=story_config
)

print("="*50)
print("【第一次创作的故事】")
print("="*50)
print(f"\n🎬 开头：\n{first_story['opening']}")
print(f"\n📖 发展：\n{first_story['development']}")
print(f"\n🎭 结局：\n{first_story['ending']}")

In [ ]:
# ========================================
# 7. 查看所有 Checkpoint（状态快照）
# ========================================

print("="*50)
print("【所有 Checkpoint 记录】")
print("="*50)

# 获取所有历史状态
all_states = list(story_graph.get_state_history(story_config))

for i, state_snapshot in enumerate(all_states):
    print(f"\n--- Checkpoint {i} ---")
    print(f"下一步节点: {state_snapshot.next}")
    print(f"Checkpoint ID: {state_snapshot.config['configurable']['checkpoint_id']}")
    
    # 显示当前状态中有值的字段
    for key, value in state_snapshot.values.items():
        if value and isinstance(value, str):
            print(f"{key}: {value[:50]}..." if len(str(value)) > 50 else f"{key}: {value}")

In [ ]:
# ========================================
# 8. 选择要回溯的 Checkpoint
# ========================================

# 找到「开头」完成后的 checkpoint（下一步是 development）
opening_checkpoint = None
for state_snapshot in all_states:
    if state_snapshot.next == ('development',):
        opening_checkpoint = state_snapshot
        break

if opening_checkpoint:
    print("【找到目标 Checkpoint】")
    print(f"Checkpoint ID: {opening_checkpoint.config['configurable']['checkpoint_id']}")
    print(f"当前状态: 开头已完成，即将生成发展")
    print(f"开头内容: {opening_checkpoint.values.get('opening', 'N/A')}")
else:
    print("未找到目标 checkpoint")

In [ ]:
# ========================================
# 9. Time Travel：从开头重新生成故事
# ========================================

if opening_checkpoint:
    # 创建回溯配置
    time_travel_config = {
        "configurable": {
            "thread_id": opening_checkpoint.config["configurable"]["thread_id"],
            "checkpoint_id": opening_checkpoint.config["configurable"]["checkpoint_id"],
        }
    }
    
    print("="*50)
    print("【Time Travel：从开头重新创作】")
    print("="*50)
    print(f"\n保留的开头：{opening_checkpoint.values.get('opening')}")
    print("\n🔄 正在生成新的故事发展和结局...\n")
    
    # 从选定的 checkpoint 继续执行
    new_story = story_graph.invoke(None, config=time_travel_config)
    
    print(f"📖 新发展：\n{new_story['development']}")
    print(f"\n🎭 新结局：\n{new_story['ending']}")

In [ ]:
# ========================================
# 10. 对比两个版本的故事
# ========================================

print("="*60)
print("【故事版本对比】")
print("="*60)

print("\n📌 共同的开头：")
print(first_story['opening'])

print("\n--- 版本 A（第一次创作）---")
print(f"发展：{first_story['development']}")
print(f"结局：{first_story['ending']}")

if opening_checkpoint:
    print("\n--- 版本 B（Time Travel 后）---")
    print(f"发展：{new_story['development']}")
    print(f"结局：{new_story['ending']}")

---

# 本节总结

## Human-in-the-Loop (HITL)

| 概念 | 说明 |
|------|------|
| `interrupt()` | 暂停 Graph 执行，等待外部输入 |
| `Command(resume=True/False)` | 恢复执行并传入决策结果 |
| `Command(goto="node_name")` | 指定下一步执行的节点 |

**适用场景**：
- 敏感操作需要人工确认
- AI 输出需要人工审核
- 关键决策需要人工参与

## Time Travel

| 概念 | 说明 |
|------|------|
| `get_state_history(config)` | 获取所有历史状态快照 |
| `checkpoint_id` | 状态快照的唯一标识 |
| 回溯执行 | 指定 checkpoint_id 从历史状态继续执行 |

**适用场景**：
- 探索不同的执行路径
- 调试和问题定位
- 版本对比和 A/B 测试

## 关键依赖

两个特性都**必须配置 Checkpointer**：

```python
from langgraph.checkpoint.memory import InMemorySaver

checkpointer = InMemorySaver()
graph = builder.compile(checkpointer=checkpointer)
```